# Creating a Standard AL workflow for water

For this tutorial we will
- create an initial starting data set for water
- set up input yaml files for an `alomancy` Standard AL Workflow object call
- run a lightweight local version of the code to perform the training
## The `StandardActiveLearningWorkflow` Object
All workflow objects are a subset of the `BaseActiveLearningWorkflow` object which essential performs the following loop


```mermaid
flowchart TD
    A((Initial Dataset)) --> B([Train MLIP Committee])
    B --> C([Structure Generation])
    C --> D[Uncertainty-based Selection]
    D --> E([High-Accuracy Evaluation])
    E --> F[Update Training Dataset]
    F --> B

    style A fill:#e66027
    style B fill:#e87322
    style C fill:#eb861e
    style D fill:#efa119
    style E fill:#708e4c
    style F fill:#328566
```

Where the round-edged nodes are required to be defiend by a subclass. 

The `StandardActiveLearningWorkflow` defines them as

- **Train MLIP Committee**: Creates a committee of from scratch [MACE](https://mace-docs.readthedocs.io/en/latest/index.html) models
- **Structure Generation**: Selects high standard deviation structures from a user defined number of [Langevin molecular dynamics (MD)](https://ase-lib.org/ase/md.html#module-ase.md.langevin) by using the committee of models
- **High-Accuracy Evaluation**: Evaluates the structuers using the plane wave density functional theory (DFT) software [Quantum Espresso (QE)](https://www.quantum-espresso.org)

With the other nodes in the flowchart being globally defined and therefore untouched here. 

Changing these nodes is covered in the next tutorial. For now we shall leave them as the `StandardActiveLearningWorkflow` defaults.

## Creating a Dataset
It is important that our initial data set contains
- All types of atoms that we wish to train upon (H and O)
- A diverse collection of potential stoichiometries, atom numbers, and cell densities

We will start with constructing atoms objects for the indiviual atoms

In [34]:
from ase import Atoms
from ase.calculators.emt import EMT

isolated_atoms_list=[]

# construct single atom Atoms objects
h_atom = Atoms('H', positions=[[0,0,0]], cell=[10,10,10], pbc=True)
o_atom = Atoms('O', positions=[[0,0,0]], cell=[10,10,10], pbc=True)

# Evaluate the objects energetically, in reality you should use a much higher accuracy method than EMT such as DFT
for atom in [h_atom, o_atom]:
    atom.calc=EMT()
    atom.get_potential_energy()
    atom.info['config_type']='IsolatedAtom'
    isolated_atoms_list.append(atom)

We assign config types to assist with the MACE training later on.

Next, we will add molecules to our data set. We will also add deformed molecules to increase the amount of information in our training set. 

In [35]:
from ase.build import molecule
from ase.optimize import BFGS

# create some locally optimisedsingle molecule objects
single_molecule_list=[]
molecule_type_list=['H2', 'O2', 'H2O', 'H2O2', 'O3']

for mol in molecule_type_list:
    mol = molecule(mol)
    mol.cell = [10, 10, 10]
    mol.pbc = True
    mol.calc = EMT()
    opt = BFGS(mol, logfile=None, trajectory=None)
    opt.run(fmax=0.05)
    mol.info['config_type'] = 'StandardMolecule'
    single_molecule_list.append(mol)


# define the deformation parameters
max_contraction = 0.3
max_expansion = 1.5
deformation_samples = 10

# create deformation multipliers
position_deformation_multipliers = [1 - max_contraction + (max_expansion - (1 - max_contraction)) * i / (deformation_samples - 1) for i in range(deformation_samples)]

# create deformed molecules
deformed_single_molecule_list = []
for mol in single_molecule_list:
    for multiplier in position_deformation_multipliers:
        deformed_mol = mol.copy()
        deformed_mol.set_positions(mol.get_positions() * multiplier)
        deformed_mol.calc = EMT()
        deformed_mol.get_potential_energy()
        deformed_mol.info['config_type'] = 'DeformedMolecule'
        deformed_single_molecule_list.append(deformed_mol)


Finally, lets generate some randomised molecular clusters so our model will know how to interpret multiple molecules

In [36]:
import numpy as np

max_number_of_molecules = 10
min_number_of_molecules = 2

random_cluster_list = []
cell_size = 20
# generate random clusters of molecules
for i in range(100):
    number_of_molecules = np.random.randint(min_number_of_molecules, max_number_of_molecules + 1)
    selected_molecule_array = np.random.choice(len(single_molecule_list), size=number_of_molecules, replace=True)
    selected_molecules = [single_molecule_list[i] for i in selected_molecule_array]

    cluster = Atoms(cell=[cell_size, cell_size, cell_size], pbc=True)
    for mol in selected_molecules:
        mol_copy = Atoms(mol.symbols, positions=mol.positions)
        # random translation
        mol_copy.set_positions(mol.positions + np.random.rand(3) * len(mol.get_positions()) * cell_size)
        # random rotation
        mol_copy.rotate(np.random.rand() * 360, 'z', center='COM')
        mol_copy.rotate(np.random.rand() * 360, 'y', center='COM')
        mol_copy.rotate(np.random.rand() * 360, 'x', center='COM')
        cluster += mol_copy

    cluster.calc = EMT()
    opt = BFGS(cluster, logfile=None, trajectory=None)
    opt.run(fmax=0.05)
    cluster.info['config_type'] = 'RandomCluster'
    cluster.info['number_of_molecules'] = number_of_molecules
    cluster.info['molecule_types'] = [mol.info['config_type'] for mol in selected_molecules]
    random_cluster_list.append(cluster)



We have now generated all structures for our initial dataset lets combine everything, then split them into test and train sets

In [37]:
all_structures = isolated_atoms_list + single_molecule_list + deformed_single_molecule_list + random_cluster_list

# assign REF_energy and REF_forces to each structure to make it easier for MACE to find
for structure in all_structures:
    structure.info['REF_energy'] = structure.get_potential_energy()
    structure.arrays['REF_forces'] = structure.get_forces()

test_train_split = int(len(all_structures) * 0.8)
train_set = all_structures[:test_train_split]
test_set = all_structures[test_train_split:]

print(f"Total structures: {len(all_structures)}")
print(f"Training set size: {len(train_set)}")
print(f"Test set size: {len(test_set)}")


Total structures: 157
Training set size: 125
Test set size: 32


This is a relatively small dataset but will suit us for our purposes for now.


## Setting-up the `ALomancy` `StandardActiveLearningWorkflow` Input

Now that we have the training data we can start setting up the input

We will use the `StandardActiveLearningWorkflow` for this. Customizing your workflow to your needs will be covered in later tutorials. 

The `StandardActiveLearningWorkflow` object takes 5 inputs

- `initial_train_file_path`: A string of the file path for the training file
- `initial_test_file_path`: A string of the file path for the test file
- `jobs_dict`: A dictionary defining all settings for the subclasses inside `StandardActiveLearningWorkflow`
- `number_of_al_loops`: The number of active learning loops one wishes to perform
- `verbose`: An integer defining how much the program talks to you (currently only 0 or 1)

As we already have made our data set in the previous version lets first write these to files

In [38]:
from ase.io import write

test_set_path= 'test_set.xyz'
train_set_path = 'train_set.xyz'

write(train_set_path, train_set, format='extxyz')
write(test_set_path, test_set, format='extxyz')

Next we will need to define our `jobs_dict`. This can be quite complicated as it needs to contain a lot of information. The structure of `jobs_dict` can be broken down into the **three** types of modules.

- `mlip_committee`: where the MLIP is trained. For the uneditted `StandardActiveLearningWorkflow` this also contains the MACE settings as `mace_fit_kwargs`
    - `name`: name of your MLIP committee job
    - `size_of_committee`: number of MLIP models to train
    - `max_time`: string of the maximum time allowed for each element in this step
    - `hpc` the high performancy computer settings associated with this calculation

- `structure_generation`: where the structures for training are generated. For the uneditted `StandardActiveLearningWorkflow` this also contains the settings for the MD as `run_md_kwargs`
    - `name`: name of your structure generation job
    - `desired_number_of_structures`: number of structures to generate
    - `max_time`: string of the maximum time allowed for each element in this step
    - `structure_selection_kwargs`: settings to decide how to select structures 
        - `max_number_of_concurrent_jobs`: number of structure generation jobs to perform (attempted concurrently depending on resources)
        - `chem_formula_list`: list of chemical formula allowed to be used in the structure selection, if all allowed set to none
        - `atom_number_range`: tuple defining the minimum and maximum numbers of atoms allowed to be used in the structure generation jobs
        - `enforce_chemical_diversity`: boolean that will ensure that no two same chemical formulas are chosen to be used in the structure generation jobs, if possible
    - `hpc` the high performancy computer settings associated with this calculation

- `high_accuracy_evaluation`: where the selected structures' energy and forces are calcualted. For the uneditted `StandardActiveLearningWorkflow` this contains the settings for the QE DFT calcualtions as `qe_input_kwargs`
    - `name`: name of your high accuracy evaluation job
    - `max_time`: string of the maximum time allowed for each element in this step
    - `hpc` the high performancy computer settings associated with this calculation

Each of these modules will need an associated high performance computer (HPC) dictionary. This dictionary defines 

- `hpc_name`: string of the ssh name of your hpc, this should be the same as the one in your `.expyre/config.json` file
- `gpu`: boolean for whether the associated submission has access to a GPU processor
- `pre_cmds`: list of commands to run before running the job, typically thesee involve activating the correct python environment
- `partitions`: list of one, partition you wish to use in your submission
- `node_info`: information about the nodes in the HPC 
    - `ranks_per_system`: number of MPIs to associate with each calculation
    - `ranks_per_node`: number of MPIs available in each node
    - `threads_per_rank`: number of OMP threads to use for each MPI
    - `max_mem_per_node`: string defining the memory to use for each node
- `high_accuracy_executable_path`: string of the path to your executable. Only needed for HPCs used for your `high_accuracy_evaluation` module and that require the calling of an executable
- `pp_path`: string of the path to your pseudopotentials directory. Only needed for HPCs used for your `high_accuracy_evaluation` module and that require pseudopotetials
- `pseudo_dict`: dictionary of elements present in the generated structures with their respective pseudopotential in the psuedopotentials directory defined by `pp_path`. Only needed for HPCs used for your `high_accuracy_evaluation` module and that require pseudopotetials


As these are quite large dictionaries it is often easier to save these settings as two configuration files either as yaml or json

Firstly, lets do the main dictionary


In [39]:
import yaml

# firstly we need the energy values of the isolated atoms to more accurately fit the model
h_energy=isolated_atoms_list[0].get_potential_energy()
o_energy=isolated_atoms_list[1].get_potential_energy()

print(f"Hydrogen energy: {h_energy:.3f} eV")
print(f"Oxygen energy: {o_energy:.3f} eV")
standard_water_config = {
  'mlip_committee': {
    'name': 'mace_committee',
    'size_of_committee': 3,
    'max_time': '5H',
    'mace_fit_kwargs': {
      'E0s': {
        1: float(h_energy),
        8: float(o_energy),
      },
      'atomic_numbers': [1, 8],
      'energy_key': "REF_energy",
      'forces_key': "REF_forces",
    },
    'hpc' : 'your_hpc_dict_0'
  },
  'structure_generation': {
    'name': 'md_1200_generation',
    'desired_number_of_structures': 50,
    'max_time': "10H",
    'structure_selection_kwargs': {
      'max_number_of_concurrent_jobs': 5,
      'chem_formula_list': None,
      'atom_number_range': [9, 21],
      'enforce_chemical_diversity': True,
    },
    'run_md_kwargs': {
      'steps': 20000,
      'temperature': 1200,
      'timestep_fs': 0.5,
      'friction': 0.002,
    },
    'hpc': 'your_hpc_dict_0',
  },
  'high_accuracy_evaluation': {
    'name': 'qe_dft',
    'max_time': "30m",
    'qe_input_kwargs': {
        'control': {
            'verbosity': "high",
            'prefix': "qe",
            'nstep': 999,
            'tstress': False,
            'tprnfor': True,
            'disk_io': "none",
            'etot_conv_thr': 1.0e-5,
            'forc_conv_thr': 1.0e-5,
            },
        'system': {
            'ibrav': 0,
            'tot_charge': 0.0,
            'ecutwfc': 40.0,
            'ecutrho': 600,
            'occupations': 'smearing',
            'degauss': 0.01,
            'smearing': 'cold',
            'input_dft': 'pbe',
            'nspin': 1,
            },
        'electrons': {
            'electron_maxstep': 999,
            'scf_must_converge': True,
            'conv_thr': 1.0e-12,
            'mixing_mode': 'local-TF',
            'mixing_beta': 0.25,
            'startingwfc': 'random',
            'diagonalization': 'david',
            },
        },
    'hpc': 'your_hpc_dict_1',
    } # the hpc can/should be changed to whatever is the most appropriate hpc for each step of the workflow
}

with open('water_standard_config.yaml', 'w') as f:
    yaml.dump(standard_water_config, f)

Hydrogen energy: 3.210 eV
Oxygen energy: 4.600 eV


Next, lets add some generic dictionaries for our hpc, you should **replace this with your own hpc data** and ensure such hpcs are correctly entered in you `.expyre/config.json`

In [40]:
your_hpc_dict = {
  'your_hpc_dict_0': {
    'hpc_name' : 'your_ssh_hpc_name',
    'gpu': True, 
    'pre_cmds': ["command to enable correct python environment on your hpc"], # e.g. `conda activate alomancy` or `source ~/.venvs/alomancy/bin/activate`
    'partitions': ["hpc_partition_name"],
    'node_info': {
      'ranks_per_system': 72, 
      'ranks_per_node': 72, 
      'threads_per_rank': 1, 
      'max_mem_per_node': "60GB", 
    },
  },
  'your_hpc_dict_1': {
    'hpc_name' : 'your_ssh_hpc_name',
    'gpu': False, 
    'pre_cmds': ["command to enable correct python environment on your hpc"], 
    'partitions': ["hpc_partition_name"],
    'node_info': {
      'ranks_per_system': 72, 
      'ranks_per_node': 72, 
      'threads_per_rank': 1, 
      'max_mem_per_node': "60GB", 
    },
    'high_accuracy_executable_path': "/path/to/your/quantum/espresso/bin/pw.x",
    'pp_path': "/path/to/your/pseudo/potentials/directory",
    'pseudo_dict': {
      'H': "name_of_hydrogen_pp.UPF",
      'O': "name_of_oxygen_pp.UPF", 
    },
  }
}

with open('hpc_config.yaml', 'w') as f:
    yaml.dump(your_hpc_dict, f)

Now lets combine these dictionaries together sensibly 

In [ ]:
from yaml import safe_load

# load jobs_dict from a YAML files
with open("water_standard_config.yaml") as f:
    config = safe_load(f)

with open("hpc_config.yaml") as f:
    hpc_config = safe_load(f)

# assign hpcs to the jobs_dict
for job in config:
    config[job]["hpc"] = hpc_config[config[job]["hpc"]]


{'high_accuracy_evaluation': {'hpc': {'gpu': False, 'high_accuracy_executable_path': '/path/to/your/quantum/espresso/bin/pw.x', 'hpc_name': 'your_ssh_hpc_name', 'node_info': {'max_mem_per_node': '60GB', 'ranks_per_node': 72, 'ranks_per_system': 72, 'threads_per_rank': 1}, 'partitions': ['hpc_partition_name'], 'pp_path': '/path/to/your/pseudo/potentials/directory', 'pre_cmds': ['command to enable correct python environment on your hpc'], 'pseudo_dict': {'H': 'name_of_hydrogen_pp.UPF', 'O': 'name_of_oxygen_pp.UPF'}}, 'max_time': '30m', 'name': 'qe_dft', 'qe_input_kwargs': {'control': {'disk_io': 'none', 'etot_conv_thr': 1e-05, 'forc_conv_thr': 1e-05, 'nstep': 999, 'prefix': 'qe', 'tprnfor': True, 'tstress': False, 'verbosity': 'high'}, 'electrons': {'conv_thr': 1e-12, 'diagonalization': 'david', 'electron_maxstep': 999, 'mixing_beta': 0.25, 'mixing_mode': 'local-TF', 'scf_must_converge': True, 'startingwfc': 'random'}, 'system': {'degauss': 0.01, 'ecutrho': 600, 'ecutwfc': 40.0, 'ibrav':

Finally we have prepared our inputs correctly and are ready to run our `StandardActiveLearningWorkflow` 

## Running `StandardActiveLearningWorkflow`

Unfortunately, not everyone has access to the same HPCs so this is quite a challenge to show in this tutorial. For now we will show you what a submission script should look like

In [ ]:
from alomancy.core.standard_active_learning import ActiveLearningStandardMACE

# define the workflow object
al_workflow = ActiveLearningStandardMACE(
    initial_train_file_path=train_set_path,
    initial_test_file_path=test_set_path,
    jobs_dict=config,
    number_of_al_loops=5,
    verbose=1,
    start_loop=0,
)

# run the workflow
al_workflow.run()

HPC: your_ssh_hpc_name, Job: mace_committee


KeyError: 'your_ssh_hpc_name'

This will, of course, fail unless you have defined your HPC dictionary properly. 

We have provided a demonstrative output in the `results` directory